In [1]:
import requests

In [2]:
from bedrock.client.client import BedrockAPI

In [3]:
SERVER = "http://localhost:81/"

In [4]:
api = BedrockAPI(SERVER)

In [5]:
filepath =  '/home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv'

In [6]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2001',ingest_id, 'default',{'file':open(filepath, "rb")})
if resp.status_code == 201:
    print 'Source {0} successfully uploaded'.format(filepath)
else:
    print 'Failed to update {0}'.format(filepath)

INFO: 2017-09-06 11:53:26,319: putting source to: http://localhost:81/dataloader/sources/Rand2001/opals.spreadsheet.Spreadsheet.Spreadsheet/default/


Source /home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv successfully uploaded


In [7]:
available_sources = api.list("dataloader","sources").json()
source = (s for s in available_sources if s["name"] == "Rand2001").next()
source_id = source["src_id"]
endpoint = api.endpoint("dataloader","sources/%s/explore/" % source_id)
resp = requests.get(endpoint)

INFO: 2017-09-06 11:53:26,540: Listing: http://localhost:81/dataloader/sources/


In [8]:
resp = api.create_matrix(source_id, 'rand_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
#resp

{u'src_id': u'efaa65941518482688656790862c001a', u'name': u'rand_mtx', u'created': u'2017-09-06 15:53:27.081622', u'outputs': [u'features_original.txt', u'features.txt', u'matrix.csv'], u'id': u'43439337415b48ee898a0e5778201d33', u'rootdir': u'/opt/bedrock/dataloader/data/efaa65941518482688656790862c001a/43439337415b48ee898a0e5778201d33/', u'filters': {u'fluid_dummy': {}, u'previous_decision': {}, u'group_size': {}, u'decision0d1c': {}, u'playerid': {}, u'_Icondition_4': {}, u'_Icondition_3': {}, u'_Icondition_2': {}, u'sessionnum': {}, u'num_neighbors': {}, u'round_num': {}, u'condition': {}}, u'mat_type': u'csv'}


In [9]:
resp = api.analytic('opals.stan.Stan.Stan_GLM')
if resp.json():
    print("Stan_GLM Opal Installed!")
else:
    print("Stan_GLM Opal Not Installed!")

Stan_GLM Opal Installed!


In [10]:
resp

<Response [200]>

## Run Stan_GLM

In [11]:
analytic_id = "opals.stan.Stan.Stan_GLM"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
            { "name" : "Formula", "attrname" : "formula", "value" : "decision0d1c ~ condition", "type" : "input" },
            { "name" : "GLM family", "attrname" : "family", "value" : 'binomial(link = "logit")', "type" : "input" },
            { "name" : "chains", "attrname" : "chains" , "value" : "3", "type" : "input"},
            { "name" : "iter", "attrname" : "iter" , "value" : "3000", "type" : "input"},
            { "name" : "prior", "attrname" : "prior" , "value" : "", "type" : "input"},
            { "name" : "prior_intercept", "attrname" : "prior_intercept" , "value" : "", "type" : "input"}
        ]

result_mtx = api.run_analytic(analytic_id, mtx, 'rand_stan_test', input_data=inputData, parameter_data=paramsData)

In [12]:
print result_mtx

None



## Get Stan_GLM outputs

In [13]:
result_mtx['src_id']

TypeError: string indices must be integers

In [ ]:
result_mtx['id']

In [ ]:
import requests

In [ ]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/prior_summary.txt/prior_summary.txt/"

In [ ]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [ ]:
response = requests.get(url)

In [ ]:
print response.text

In [ ]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/summary.txt/summary.txt/"

In [ ]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [ ]:
response2 = requests.get(url)

In [ ]:
print response2.text